Простой пример: Обработка запросов по уровню важности
Представим систему поддержки, где запросы обрабатываются разными уровнями в зависимости от их сложности (Level 1, Level 2, Boss). Каждый уровень пытается обработать запрос, и если не может, передаёт его выше.

In [1]:
from abc import ABC, abstractmethod
from typing import Any, Optional

# 1. Handler Interface (Интерфейс Обработчика)
class Handler(ABC):
    """Абстрактный обработчик."""
    def __init__(self):
        self._successor: Optional[Handler] = None

    def set_next(self, handler: 'Handler') -> 'Handler':
        """Устанавливает следующего обработчика в цепочке."""
        self._successor = handler
        # Возвращаем следующего, чтобы можно было строить цепочку вызовов:
        # handler1.set_next(handler2).set_next(handler3)
        return handler

    @abstractmethod
    def handle_request(self, request_level: int) -> Optional[str]:
        """Обрабатывает запрос или передает его дальше."""
        pass

    def _pass_to_next(self, request_level: int) -> Optional[str]:
        """Вспомогательный метод для передачи запроса."""
        if self._successor:
            print(f"  Passing request level {request_level} from {type(self).__name__} to {type(self._successor).__name__}")
            return self._successor.handle_request(request_level)
        else:
            print(f"  End of chain reached at {type(self).__name__}. Request level {request_level} cannot be handled.")
            return None # Или вернуть сообщение об ошибке по умолчанию

# 2. Concrete Handlers (Конкретные Обработчики)
class Level1Support(Handler):
    """Поддержка первого уровня - обрабатывает простые запросы (<= 1)."""
    def handle_request(self, request_level: int) -> Optional[str]:
        print(f"Level1Support checking request level {request_level}...")
        if request_level <= 1:
            result = f"Level 1 Support handled the simple request (level {request_level})."
            print(f"  {result}")
            return result
        else:
            print(f"  Level 1 Support cannot handle level {request_level}.")
            return self._pass_to_next(request_level)

class Level2Support(Handler):
    """Поддержка второго уровня - обрабатывает средние запросы (<= 2)."""
    def handle_request(self, request_level: int) -> Optional[str]:
        print(f"Level2Support checking request level {request_level}...")
        if request_level <= 2:
            result = f"Level 2 Support handled the moderate request (level {request_level})."
            print(f"  {result}")
            return result
        else:
            print(f"  Level 2 Support cannot handle level {request_level}.")
            return self._pass_to_next(request_level)

class BossSupport(Handler):
    """Начальник - обрабатывает сложные запросы (<= 3)."""
    def handle_request(self, request_level: int) -> Optional[str]:
        print(f"BossSupport checking request level {request_level}...")
        if request_level <= 3:
            result = f"The Boss handled the complex request (level {request_level})."
            print(f"  {result}")
            return result
        else:
            print(f"  The Boss cannot handle level {request_level} (too complex?).")
            # У Босса нет следующего, он не вызывает _pass_to_next, а возвращает None/ошибку
            return f"Request level {request_level} is too complex, even for the Boss!"

# 3. Client Code (Клиентский Код)
def client_code(handler: Handler, request_level: int):
    """Клиент отправляет запрос первому обработчику в цепочке."""
    print(f"\nClient: Sending request with level {request_level}...")
    result = handler.handle_request(request_level)
    if result:
        print(f"Client: Request processed: {result}")
    else:
        print(f"Client: Request could not be processed by the chain.")

if __name__ == "__main__":
    # Собираем цепочку
    level1 = Level1Support()
    level2 = Level2Support()
    boss = BossSupport()

    level1.set_next(level2).set_next(boss) # Строим цепочку: level1 -> level2 -> boss

    # Отправляем разные запросы
    client_code(level1, 1) # Должен обработать Level 1
    client_code(level1, 2) # Должен обработать Level 2
    client_code(level1, 3) # Должен обработать Boss
    client_code(level1, 4) # Никто не сможет обработать

# Вывод:
# Client: Sending request with level 1...
# Level1Support checking request level 1...
#   Level 1 Support handled the simple request (level 1).
# Client: Request processed: Level 1 Support handled the simple request (level 1).
#
# Client: Sending request with level 2...
# Level1Support checking request level 2...
#   Level 1 Support cannot handle level 2.
#   Passing request level 2 from Level1Support to Level2Support
# Level2Support checking request level 2...
#   Level 2 Support handled the moderate request (level 2).
# Client: Request processed: Level 2 Support handled the moderate request (level 2).
#
# Client: Sending request with level 3...
# Level1Support checking request level 3...
#   Level 1 Support cannot handle level 3.
#   Passing request level 3 from Level1Support to Level2Support
# Level2Support checking request level 3...
#   Level 2 Support cannot handle level 3.
#   Passing request level 3 from Level2Support to BossSupport
# BossSupport checking request level 3...
#   The Boss handled the complex request (level 3).
# Client: Request processed: The Boss handled the complex request (level 3).
#
# Client: Sending request with level 4...
# Level1Support checking request level 4...
#   Level 1 Support cannot handle level 4.
#   Passing request level 4 from Level1Support to Level2Support
# Level2Support checking request level 4...
#   Level 2 Support cannot handle level 4.
#   Passing request level 4 from Level2Support to BossSupport
# BossSupport checking request level 4...
#   The Boss cannot handle level 4 (too complex?).
# Client: Request processed: Request level 4 is too complex, even for the Boss!


Client: Sending request with level 1...
Level1Support checking request level 1...
  Level 1 Support handled the simple request (level 1).
Client: Request processed: Level 1 Support handled the simple request (level 1).

Client: Sending request with level 2...
Level1Support checking request level 2...
  Level 1 Support cannot handle level 2.
  Passing request level 2 from Level1Support to Level2Support
Level2Support checking request level 2...
  Level 2 Support handled the moderate request (level 2).
Client: Request processed: Level 2 Support handled the moderate request (level 2).

Client: Sending request with level 3...
Level1Support checking request level 3...
  Level 1 Support cannot handle level 3.
  Passing request level 3 from Level1Support to Level2Support
Level2Support checking request level 3...
  Level 2 Support cannot handle level 3.
  Passing request level 3 from Level2Support to BossSupport
BossSupport checking request level 3...
  The Boss handled the complex request (le

Сложный пример: Обработка событий в GUI (упрощенная имитация)
Представим иерархию виджетов (Кнопка -> Панель -> Окно). Событие (например, клик) сначала пытается обработать самый вложенный виджет (Кнопка). Если он не обрабатывает, событие "всплывает" к родителю (Панель), и так далее.

In [2]:
from abc import ABC, abstractmethod
from typing import Optional, Dict, Any

# --- Событие ---
class Event:
    def __init__(self, name: str, data: Optional[Dict[str, Any]] = None):
        self.name = name
        self.data = data or {}
        self.handled = False # Флаг, что событие уже обработано

    def __str__(self):
        return f"Event(name='{self.name}', handled={self.handled}, data={self.data})"

# --- 1. Handler Interface (Компонент GUI) ---
class UIComponent(ABC):
    """Абстрактный компонент GUI, может обрабатывать события."""
    def __init__(self, name: str, parent: Optional['UIComponent'] = None):
        self._name = name
        self._parent = parent # Ссылка на родителя (это и есть "следующий" в цепочке всплытия)

    @property
    def name(self) -> str:
        return self._name

    def handle_event(self, event: Event):
        """
        Пытается обработать событие. Если не может, передает родителю.
        Используем флаг event.handled для остановки цепочки.
        """
        if not event.handled:
            print(f"{self.name}: Checking event {event.name}...")
            if self._can_handle(event):
                self._process_event(event)
                # Можно установить флаг handled, если событие не должно всплывать дальше
                # event.handled = True
            else:
                print(f"{self.name}: Cannot handle '{event.name}', passing to parent.")
                if self._parent:
                    self._parent.handle_event(event) # Передаем родителю
                else:
                    print(f"{self.name}: No parent to pass event '{event.name}' to. Event dropped.")

    @abstractmethod
    def _can_handle(self, event: Event) -> bool:
        """Проверяет, может ли этот компонент обработать событие."""
        pass

    @abstractmethod
    def _process_event(self, event: Event) -> None:
        """Выполняет обработку события."""
        pass

# --- 2. Concrete Handlers (Конкретные Виджеты) ---
class Button(UIComponent):
    def _can_handle(self, event: Event) -> bool:
        # Кнопка может обработать событие 'click'
        return event.name == 'click'

    def _process_event(self, event: Event) -> None:
        print(f"*** {self.name}: Clicked! Performing button action... ***")
        event.handled = True # Кнопка обработала клик, дальше передавать не нужно

class Panel(UIComponent):
    def __init__(self, name: str, parent: Optional['UIComponent'] = None, has_context_menu: bool = False):
        super().__init__(name, parent)
        self._has_context_menu = has_context_menu

    def _can_handle(self, event: Event) -> bool:
        # Панель может обработать 'right_click' (если есть контекстное меню)
        # или событие 'resize'
        return (event.name == 'right_click' and self._has_context_menu) or \
               event.name == 'resize'

    def _process_event(self, event: Event) -> None:
        if event.name == 'right_click':
            print(f"*** {self.name}: Right-clicked! Showing context menu... ***")
            event.handled = True
        elif event.name == 'resize':
            print(f"*** {self.name}: Resized! Adjusting layout... ***")
            # Resize может быть обработан и панелью, и окном - не ставим handled=True
            # event.handled = False # Явно оставляем возможность всплытия

class Window(UIComponent):
    def _can_handle(self, event: Event) -> bool:
        # Окно может обработать 'close' или 'resize'
        return event.name == 'close' or event.name == 'resize'

    def _process_event(self, event: Event) -> None:
        if event.name == 'close':
            print(f"*** {self.name}: Closed! Saving state and exiting... ***")
            event.handled = True
        elif event.name == 'resize':
            print(f"*** {self.name}: Resized! Adjusting window size... ***")
            # Окно - последний обработчик, можно поставить handled
            event.handled = True

# --- 3. Client Code ---
if __name__ == "__main__":
    # Собираем иерархию (цепочку)
    main_window = Window("Main App Window")
    content_panel = Panel("Content Panel", parent=main_window, has_context_menu=True)
    ok_button = Button("OK Button", parent=content_panel)
    cancel_button = Button("Cancel Button", parent=content_panel)
    status_panel = Panel("Status Panel", parent=main_window) # Панель без контекстного меню

    # Имитируем события, отправляя их самому внутреннему компоненту
    # (или тому, где произошло событие)

    print("\n--- Simulating Click on OK Button ---")
    click_event = Event('click')
    ok_button.handle_event(click_event)
    print(f"Event after chain: {click_event}")

    print("\n--- Simulating Right Click on Content Panel ---")
    right_click_event = Event('right_click')
    content_panel.handle_event(right_click_event) # Отправляем панели
    print(f"Event after chain: {right_click_event}")

    print("\n--- Simulating Right Click on Status Panel (no menu) ---")
    right_click_event_status = Event('right_click')
    status_panel.handle_event(right_click_event_status) # Отправляем панели статуса
    print(f"Event after chain: {right_click_event_status}") # Должно всплыть до окна и пропасть

    print("\n--- Simulating Resize Event on Panel ---")
    resize_event = Event('resize')
    # Отправляем панели, но она может передать дальше (если не ставит handled=True)
    content_panel.handle_event(resize_event)
    print(f"Event after chain: {resize_event}")

    print("\n--- Simulating Close Event on Window ---")
    close_event = Event('close')
    main_window.handle_event(close_event) # Отправляем прямо окну
    print(f"Event after chain: {close_event}")

# Вывод:
# --- Simulating Click on OK Button ---
# OK Button: Checking event click...
# *** OK Button: Clicked! Performing button action... ***
# Event after chain: Event(name='click', handled=True, data={})
#
# --- Simulating Right Click on Content Panel ---
# Content Panel: Checking event right_click...
# *** Content Panel: Right-clicked! Showing context menu... ***
# Event after chain: Event(name='right_click', handled=True, data={})
#
# --- Simulating Right Click on Status Panel (no menu) ---
# Status Panel: Checking event right_click...
# Status Panel: Cannot handle 'right_click', passing to parent.
# Main App Window: Checking event right_click...
# Main App Window: Cannot handle 'right_click', passing to parent.
# Main App Window: No parent to pass event 'right_click' to. Event dropped.
# Event after chain: Event(name='right_click', handled=False, data={})
#
# --- Simulating Resize Event on Panel ---
# Content Panel: Checking event resize...
# *** Content Panel: Resized! Adjusting layout... ***
# Content Panel: Passing event resize to parent (because handled=False implicitly)
# Main App Window: Checking event resize...
# *** Main App Window: Resized! Adjusting window size... ***
# Event after chain: Event(name='resize', handled=True, data={})
#
# --- Simulating Close Event on Window ---
# Main App Window: Checking event close...
# *** Main App Window: Closed! Saving state and exiting... ***
# Event after chain: Event(name='close', handled=True, data={})


--- Simulating Click on OK Button ---
OK Button: Checking event click...
*** OK Button: Clicked! Performing button action... ***
Event after chain: Event(name='click', handled=True, data={})

--- Simulating Right Click on Content Panel ---
Content Panel: Checking event right_click...
*** Content Panel: Right-clicked! Showing context menu... ***
Event after chain: Event(name='right_click', handled=True, data={})

--- Simulating Right Click on Status Panel (no menu) ---
Status Panel: Checking event right_click...
Status Panel: Cannot handle 'right_click', passing to parent.
Main App Window: Checking event right_click...
Main App Window: Cannot handle 'right_click', passing to parent.
Main App Window: No parent to pass event 'right_click' to. Event dropped.
Event after chain: Event(name='right_click', handled=False, data={})

--- Simulating Resize Event on Panel ---
Content Panel: Checking event resize...
*** Content Panel: Resized! Adjusting layout... ***
Event after chain: Event(name='